In [1]:
from collections import namedtuple
import os
import tqdm
os.chdir("../")

In [2]:
from collections import namedtuple
from classifier_app.constants import *
from classifier_app.utils import read_yaml,create_directories
import os
import urllib.request as request
from zipfile import ZipFile

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksconfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    early_stopping_checkpoint_filepath: Path

In [4]:


class ConfigurationManager:


    def __init__(self, 
                    config_file_path = CONFIG_FILE_PATH,
                    params_filepath = PARAMS_FILE_PATH):
                    
                    self.config = read_yaml(config_file_path)
                    self.params = read_yaml(params_filepath)
                    
                    create_directories([Path(self.config.artifacts_root)])

    def get_prepareCallbackConfig(self)-> PrepareCallbacksconfig:
        config = self.config.prepare_callbacks
        params = self.params


        # checkpoint_model_dir, _  =  os.path.split(config.checkpoint_model_filepath)
        checkpoint_model_dir = os.path.dirname(config.checkpoint_model_filepath)
        early_stopping_checkpoint_dir = os.path.dirname(config.early_stopping_checkpoint_filepath)

        
        create_directories([Path(config.root_dir), Path(config.tensorboard_root_log_dir),
                            Path(checkpoint_model_dir),
                            Path(early_stopping_checkpoint_dir)])
        

        prepareCallbacksconfig = PrepareCallbacksconfig(
                                root_dir=Path(config.root_dir),
                                tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                                checkpoint_model_filepath=Path(config.checkpoint_model_filepath),
                                early_stopping_checkpoint_filepath= Path(config.early_stopping_checkpoint_filepath)
                                
                                )
                                
        return prepareCallbacksconfig    

In [7]:
import tensorflow as tf
import time

class PrepareCallback:

    def __init__(self, config: PrepareCallbacksconfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
                                        self.config.tensorboard_root_log_dir,
                                        f"tb_logs_at_{timestamp}"
        )
        print(tb_running_log_dir)                                
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only= True
        )

    @property
    def _create_earlyStopping_callbacks(self):
        return tf.keras.callbacks.EarlyStopping(
            monitor= "val_loss",
            min_delta= 0.001,
            patience=8,
            verbose = 0,
            mode = "auto",
            restore_best_weights=True            
        )


    def get_tb_ckpt_estopping_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks,
            self._create_earlyStopping_callbacks
        ]



In [8]:
try :

    config = ConfigurationManager()
    prepareCallbacks = PrepareCallback(config= config.get_prepareCallbackConfig())
    prepareCallbacks.get_tb_ckpt_estopping_callbacks()

except Exception as e:
    raise e

artifacts\prepare_callbacks\tensorboard_log_dir\tb_logs_at_2022-10-28-17-19-00


In [ ]:
import time

time.strftime("%Y-%m-%d-%H-%M-%S")

'2022-10-28-17-12-27'